In [ ]:
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
from sklearn import metrics
import numpy as np
import argparse
import locale
import os
import import_ipynb
import house_models
import numrical_features
import images_features

# construct the path to the input .txt file 
print("[INFO] loading house attributes...")
inputPath = "../HousesInfo.txt"
df = numrical_features.load_house_attributes(inputPath)

# load the house images
print("[INFO] loading house images...")
images = images_features.load_house_images(df, "../.")
images = images / 255.0 #normalization-scale the pixel intensities to the range [0, 1]

# split the data into training(75%) and testing(25%)
print("[INFO] processing data...")
(trainAttrX, testAttrX, trainImagesX, testImagesX) = train_test_split(df, images, test_size=0.25, random_state=42)

#check for var test in compare to test error
avg_train=trainAttrX["price"].mean()
print("avg price on the train: ", avg_train)
var_test=0
var_test=((testAttrX["price"]-avg_train)**2).mean()

# scale our house prices to the range [0, 1] 
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice
 
# performing min-max scaling on continuous features,
# one-hot encoding on categorical features,
# concatenating them together.
(trainAttrX, testAttrX) = numrical_features.process_house_attributes(df,trainAttrX, testAttrX)

# create the MLP and CNN models
mlp = house_models.create_mlp(trainAttrX.shape[1])
cnn = house_models.create_cnn(64, 64, 3)

# combine the output of the MLP and CNN together 
combinedInput = concatenate([mlp.output, cnn.output])
 
# our final fully connected layer head will have three dense layers
x = Dense(8, activation="relu")(combinedInput)
x = Dense(4, activation="relu")(x)
x = Dense(1, activation="linear")(x)
 
# input: categorical/numerical data on the MLP and images on the CNN,
# outputting: predicted price
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

# compile the model using mean_squared_error as our loss
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_error", optimizer=opt)
 
# train the model
print("[INFO] training model...")
model.fit([trainAttrX, trainImagesX], trainY, validation_data=([testAttrX, testImagesX], testY), epochs=200, batch_size=8)
 
# predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict([testAttrX, testImagesX])

# preprocessing before mean squared error
testY = testY*maxPrice
preds = preds*maxPrice
np_testY=np.array(testY)
np_preds=np.array(preds)
print("Real price of house 305:",np_testY[0])
print("Price prediction of house 305:",np_preds[0])

# mean squared error
print("average model result: ",var_test)
print('mean squared error: ',metrics.mean_squared_error(np_testY,np_preds))

[INFO] loading house attributes...
[INFO] loading house images...
[INFO] processing data...
avg price on the train:  524290.0
[INFO] training model...
Epoch 1/200
34/34 [==============================] - 3s 91ms/step - loss: 0.1927 - val_loss: 0.0459
Epoch 2/200
34/34 [==============================] - 3s 78ms/step - loss: 0.0633 - val_loss: 0.0520
Epoch 3/200
34/34 [==============================] - 3s 78ms/step - loss: 0.0530 - val_loss: 0.0473
Epoch 4/200
34/34 [==============================] - 3s 78ms/step - loss: 0.0496 - val_loss: 0.0494
Epoch 5/200
34/34 [==============================] - 3s 79ms/step - loss: 0.0459 - val_loss: 0.0470
Epoch 6/200
34/34 [==============================] - 3s 79ms/step - loss: 0.0455 - val_loss: 0.0444
Epoch 7/200
34/34 [==============================] - 3s 78ms/step - loss: 0.0436 - val_loss: 0.0456
Epoch 8/200
34/34 [==============================] - 3s 78ms/step - loss: 0.0408 - val_loss: 0.0488
Epoch 9/200
34/34 [==============================

34/34 [==============================] - 2s 73ms/step - loss: 0.0175 - val_loss: 0.0181
Epoch 81/200
34/34 [==============================] - 2s 66ms/step - loss: 0.0167 - val_loss: 0.0199
Epoch 82/200
34/34 [==============================] - 2s 68ms/step - loss: 0.0166 - val_loss: 0.0192
Epoch 83/200
34/34 [==============================] - 2s 70ms/step - loss: 0.0168 - val_loss: 0.0179
Epoch 84/200
34/34 [==============================] - 2s 60ms/step - loss: 0.0173 - val_loss: 0.0184
Epoch 85/200
34/34 [==============================] - ETA: 0s - loss: 0.016 - 2s 67ms/step - loss: 0.0165 - val_loss: 0.0185
Epoch 86/200
34/34 [==============================] - 2s 65ms/step - loss: 0.0172 - val_loss: 0.0179
Epoch 87/200
34/34 [==============================] - 2s 62ms/step - loss: 0.0177 - val_loss: 0.0186
Epoch 88/200
34/34 [==============================] - 2s 61ms/step - loss: 0.0175 - val_loss: 0.0199
Epoch 89/200
34/34 [==============================] - 2s 62ms/step - loss: 0.017